<a href="https://colab.research.google.com/github/nshoo/Neural-Synthesis-Visualization/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

## Setup

In [ ]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Prepare the data

In [ ]:
#  Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()


# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train_copy = np.zeros(shape=(13007, 28,28,1))
x_test_copy = np.zeros(shape=(2163,28,28,1))
y_train_copy = np.zeros(shape=(13007, 10))
y_test_copy = np.zeros(shape=(2163, 10))

n = len(x_train)
index = 0
for i in range(n):
  if(y_train[(i,1)] == 1.0 or y_train[(i,7)] == 1.0):
    x_train_copy[index] = x_train[i]
    y_train_copy[index] = y_train[i]
    index += 1

n = len(x_test)
index2 = 0
for i in range(n):
  if(y_test[(i,1)] == 1.0 or y_test[(i,7)] == 1.0):
    x_test_copy[index2] = x_test[i]
    y_test_copy[index2] = y_test[i]
    index2 += 1

print("x_train shape:", x_train_copy.shape)
print(x_train_copy.shape[0], "train samples")
print(x_test_copy.shape[0], "test samples")

#print("x_train: ", x_train_copy[0]) # shape (28,28,1) which means triple array,
                                    # whihc means there is one big array
                                    # where there are 28 arrays containing 28
                                    # arrays of size 1 (one value per array)

                                    # [0, 0, 0]
#print("y_train: ", y_train_copy[0]) # shape (10)
#print("x_test: ", x_test_copy[0]) # shape (28,28,1)
#print("y_test: ", y_test_copy[0]) # shape (10)

test_run_test = np.zeros(shape=784)

# fill in the test array
index3 = 0
# print(input_shape[0])
for j in range(input_shape[0]):
  for k in range(input_shape[1]):
    test_run_test[index3] = x_test_copy[0][j][k][0]
    index3 += 1
print("The number of pixels in the image = " + str(index3))

print(test_run_test)

print(x_train_copy[0].flatten())

def display_digit(img):
  for row in img:
    for pix in row:
      print('█' if pix > 0 else ' ', end='')
    print('')

print(display_digit(x_train_copy[0]))
print(y_train_copy[0])

11501568/11490434 [==============================] - 0s 0us/step
x_train shape: (13007, 28, 28, 1)
13007 train samples
2163 test samples
The number of pixels in the image = 784
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

## Build the model

In [ ]:
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

## Train the model

In [ ]:
batch_size = 128
# Normally 15
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train_copy, y_train_copy, batch_size=batch_size, epochs=epochs, validation_split=0.1)

model.save('my_model')

Epoch 1/15
92/92 [==============================] - 11s 114ms/step - loss: 0.2575 - accuracy: 0.9451 - val_loss: 0.0143 - val_accuracy: 0.9969
Epoch 2/15
92/92 [==============================] - 10s 111ms/step - loss: 0.0241 - accuracy: 0.9927 - val_loss: 0.0078 - val_accuracy: 0.9985
Epoch 3/15
92/92 [==============================] - 10s 110ms/step - loss: 0.0169 - accuracy: 0.9953 - val_loss: 0.0052 - val_accuracy: 0.9977
Epoch 4/15
92/92 [==============================] - 10s 109ms/step - loss: 0.0136 - accuracy: 0.9956 - val_loss: 0.0050 - val_accuracy: 0.9985
Epoch 5/15
92/92 [==============================] - 10s 110ms/step - loss: 0.0125 - accuracy: 0.9956 - val_loss: 0.0040 - val_accuracy: 0.9985
Epoch 6/15
92/92 [==============================] - 10s 110ms/step - loss: 0.0113 - accuracy: 0.9965 - val_loss: 0.0038 - val_accuracy: 0.9992
Epoch 7/15
92/92 [==============================] - 10s 111ms/step - loss: 0.0095 - accuracy: 0.9970 - val_loss: 0.0053 - val_accuracy: 0.9985

## Evaluate the trained model

In [ ]:
import math
from collections import Counter

def display_digit(img):
  for row in img:
    for pix in row:
      print('█' if pix > 0 else ' ', end='')
    print('')

def get_truth(arr):
  for i, value in enumerate(arr):
    if value > 0:
      return i
  return -1

def get_prediction(arr):
  max_digit = 0
  for i, value in enumerate(arr):
    if arr[max_digit] < value:
      max_digit = i
  return max_digit

# The ground truth value (which digit) associated with each test case
ground_truth = [get_truth(y) for y in y_test_copy]
# The number of times each digit appears in the tests (worth noting that each
# digit does not occur the same number of times)
test_occur = Counter(ground_truth)

def distance(arr1, arr2):
  return math.sqrt(sum([pow(arr1[i] - arr2[i], 2) for i in range(len(arr1))]))

# The outputs of the model for each test case
predictions = model.predict(x_test_copy)
# The accuracy of the model's predictions (lower score is better)
scores = [distance(predictions[i], y_test_copy[i]) for i in range(len(predictions))]
# The indicies of the "predictions" array sorted in descending order by score
# The ones that come first point to tests for which the model was least accurate
offending_inds = list(reversed(sorted([i for i in range(len(predictions))], key=lambda x: scores[x])))
# The "hardest" tests mapped to their ground truth values
offending_digits = list(map(lambda x: ground_truth[x], offending_inds))

# The average score for each digit, 0-9 (lower is better)
avg_scores = {1: 0, 7: 0}
for ind in offending_inds:
  avg_scores[offending_digits[ind]] += scores[ind]
for key in avg_scores:
  avg_scores[key] /= test_occur[key]

ranked_digits = reversed(sorted(avg_scores.keys(), key=lambda x: avg_scores[x]))
print("Average score for each digit:")
print("(lower is better)".center(29))
print("=============================")

for digit in ranked_digits:
  print("{}:  {}".format(digit, round(avg_scores[digit], 5)).center(29))
print('')

print("The first ten most challenging test cases:")
print("==========================================")

for ind in offending_inds[:10]:
  print("Digit: {}".format(ground_truth[ind]))
  print("Prediction: {}".format(get_prediction(predictions[ind])))
  print("Score: {}".format(round(scores[ind], 5)))
  print("Index: x_test[{}]".format(ind))
  display_digit(x_test_copy[ind])


# Identify test cases close to decision boundaries
def get_choices(pred):
  highest_inds = sorted([i for i in range(len(pred))], key=lambda x: pred[x])
  return list(reversed(highest_inds[-2:]))

def get_indecision(pred):
  highest = sorted(pred)[-2:]
  return highest[1] - highest[0]

closest = list(sorted([i for i in range(len(predictions))], key=lambda x: get_indecision(predictions[x])))


#print(ground_truth)

for ind in closest[:10]:
  print("Choices:", get_choices(predictions[ind]))
  print("Predicted:", get_prediction(predictions[ind]))
  print("Assigned (ground truth):", ground_truth[ind])
  print([round(x, 4) for x in predictions[ind]])
  print("===============================")
  print('')

  display_digit(x_test_copy[ind])
  print('')



perturbed = np.copy(x_test_copy[closest[1]])
original_pred = model.predict(np.array([perturbed,]))[0]
original_digit = get_prediction(original_pred)

def toggle(img, ind):
  img[ind] = np.float32(0.0 if img[ind] > 0 else 1.0)


gen = 0
found = False
while True:
  confused_coord = None
  confused_indecision = 0

  for y, row in enumerate(perturbed):
    if found: break
    for x, pix in enumerate(row):
      toggle(perturbed, (y, x))
      pred = model.predict(np.array([perturbed,]))[0]
      
      digit = get_prediction(pred)
      if digit != original_digit:
        print("New: {} - ({}, {})".format(digit, x, y))
        found = True
        break

      indecision = get_indecision(pred)
      if confused_coord == None or indecision < confused_indecision:
        confused_coord = (x, y)
        confused_indecision = indecision
      toggle(perturbed, (y, x))
    
  
  print('')

  if found:
    print("Original: {}".format(original_digit))
    print(original_pred)
    display_digit(perturbed)
    print("Decision boundary reached! ^")
    break

  toggle(perturbed, (confused_coord[1], confused_coord[0]))
  display_digit(perturbed)
  print("Generation: {} ^".format(gen))
  gen += 1






Average score for each digit:
      (lower is better)      
         1:  0.00404         
          7:  0.002          

The first ten most challenging test cases:
Digit: 1
Prediction: 7
Score: 1.31875
Index: x_test[662]
                            
                            
                            
                            
             ███            
           █████            
         ████████           
       ██████████           
      ███████████           
      █████  ████           
      █████  ████           
      ██     ████           
              ███           
              ███           
              ███           
              ███    ████   
              ████████████  
               ███████████  
             █████████████  
          ██████████        
        ███████████         
      █████████████         
      ███████   ███         
      █████                 
                            
                            
                         

In [ ]:
counter = 0
g = open("allConstraints.txt", "w")

for ind in closest:
  digit = x_test_copy[ind]
  # for when we need ground truth values
  # truth = ground_truth[ind]
  truth = get_choices(predictions[ind])[0]
  g.write("(constraint (= (rig_mimic {}) {}))\n".format(' '.join(map(lambda x: "{:.4f}".format(x), digit.flatten())), 'true' if truth == 7 else 'false'))
  counter += 1

g.close()

print(counter)

2163


In [ ]:
# Generating 100 examples for SYNT

num = 25
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
n = len(predictions)
# print(n)
j = 0
k = 0

for i in range(0, n):
    if get_choices(predictions[i])[0] == 1:
        j += 1
    elif get_choices(predictions[i])[0] == 7:
        k += 1
ones = np.zeros(shape=(j,784)) #.astype(int)
sevens = np.zeros(shape=(k,784)) #.astype(int)


# print("j: ", j)
# print("k: ", k)

a = 0
b = 0
for i in range(0, n):
    if get_choices(predictions[i])[0] == 1:
        flat = x_test_copy[i].flatten()
        ones[a] = flat
        a += 1
    elif get_choices(predictions[i])[0] == 7:
        flat = x_test_copy[i].flatten()
        sevens[b] = flat
        b += 1

onesFile = [ones[i].astype(int) for i in range(len(ones))]
sevensFile = [sevens[i].astype(int) for i in range(len(sevens))]

np.random.shuffle(ones)
np.random.shuffle(sevens)
# used to track the index of the previous digit
oneForSeven = [-1]*num
closestOnes = [0]*num
for i in range(num):
    neighbors = [[0,-1] for _ in range(len(ones)-1)]
    count = 0
    for j in range(0, len(ones)):
        if i != j:
            neighbors[count][1] = j
            neighbors[count][0] = cosine_similarity(ones[i].reshape(1, -1), ones[j].reshape(1, -1)).flatten().tolist()
            neighbors[count][0] = neighbors[count][0][0]
            count += 1
    max = -1000
    ind = 0
    for k in range(len(neighbors)):
        if max < neighbors[k][0] and (neighbors[k][1] not in oneForSeven):
          max = neighbors[k][0]
          ind = k
    oneForSeven[i] = neighbors[ind][1] # jth index
    pair = tuple((ones[i], ones[neighbors[ind][1]])) # ith and jth images
    closestOnes[i] = pair




def display_digit2(img):
  for i in range(28):
    for j in range(28):
      print('█' if img[j + 28*i] > 0 else ' ', end='')
    print('')

for i in range(num):
    print(cosine_similarity(closestOnes[i][0].reshape(1,-1),closestOnes[i][1].reshape(1,-1)).flatten())
    print(display_digit2(closestOnes[i][0]))
    print(display_digit2(closestOnes[i][1]))

[0.96262445]
                            
                            
                            
                            
             ███            
             ████           
             ████           
             ████           
             ████           
             ████           
             ████           
             ████           
             ████           
             ████           
             ████           
             ███            
            ████            
            ████            
            ████            
            ████            
            ████            
            ██████          
             █████          
             ████           
                            
                            
                            
                            
None
                            
                            
                            
                            
            ████            
            ████         

In [ ]:
sevenForSeven = [-1]*num
closestSevensToOnes = [0]*num
for i in range(num):
  oneToSevens = [[0,-1] for _ in range(len(sevens)-1)]
  count1 = 0
  for j in range(0, len(sevens) - 1):
      oneToSevens[count1][1] = j
      oneToSevens[count1][0] = cosine_similarity(ones[oneForSeven[i]].reshape(1,-1), sevens[j].reshape(1,-1)).flatten().tolist()
      oneToSevens[count1][0] = oneToSevens[count1][0][0]
      count1 += 1
  max1 = 0
  ind1 = 0
  for k in range(len(oneToSevens)):
    if max1 < oneToSevens[k][0] and (oneToSevens[k][1] not in sevenForSeven):
      max1 = oneToSevens[k][0]
      ind1 = k
  sevenForSeven[i] = oneToSevens[ind1][1]
  pair1 = tuple((ones[oneForSeven[i]], sevens[oneToSevens[ind1][1]]))
  closestSevensToOnes[i] = pair1


print(closestSevensToOnes[0])
for i in range(num):
    print(cosine_similarity(closestSevensToOnes[i][0].reshape(1,-1),closestSevensToOnes[i][1].reshape(1,-1)).flatten())
    print(display_digit2(closestSevensToOnes[i][0]))
    print(display_digit2(closestSevensToOnes[i][1]))

(array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.    

In [ ]:


closestSevensToSevens = [0]*num
usedIndices = [-1]*num
for i in range(num):
  sevenToSevens = [[0,-1] for _ in range(len(sevens)-1)]#[[0]*3]*len(sevens)
  count2 = 0
  for j in range(0, len(sevens)):
    if sevenForSeven[i] != j:
      sevenToSevens[count2][1] = j
      sevenToSevens[count2][0] = cosine_similarity(sevens[sevenForSeven[i]].reshape(1,-1), sevens[j].reshape(1,-1)).flatten().tolist()
      sevenToSevens[count2][0] = sevenToSevens[count2][0][0]
      count2 += 1
  max2 = -1000
  ind2 = 0
  for k in range(len(sevenToSevens)):
    if max2 < sevenToSevens[k][0] and (sevenToSevens[k][1] not in usedIndices):
      max2 = sevenToSevens[k][0]
      ind2 = k
  usedIndices[i] = sevenToSevens[ind2][1]
  pair2 = tuple((sevens[sevenForSeven[i]], sevens[sevenToSevens[ind2][1]]))
  closestSevensToSevens[i] = pair2

onesFile = [ones[i].astype(int) for i in range(len(ones))]
otsFile = [closestSevensToOnes[i][0].astype(int) for i in range(len(closestSevensToOnes))]
stsFile = [closestSevensToSevens[i][0].astype(int) for i in range(len(closestSevensToSevens))]
sts2File = [closestSevensToSevens[i][1].astype(int) for i in range(len(closestSevensToSevens))]



In [ ]:
import json

# for i in range(100):
w = [ones[i] for i in range(25)]
x = [closestSevensToOnes[i][0] for i in range(25)]
y = [closestSevensToSevens[i][0] for i in range(25)]
z = [closestSevensToSevens[i][1] for i in range(25)]

hundredArray = [w, x, y, z]
print(hundredArray)

# g = open("array.txt", "w")
# g.write(str(hundredArray))
# g.close()
  

f = open("constraints.smt2", "w")

for i in range(num):
  digit_one = ones[i]
  f.write("(constraint (= (rig_mimic {}) {}))\n".format(' '.join(map(lambda x: "{:.4f}".format(x), digit_one.flatten())), 'false'))

for i in range(num):
  digit_two = closestSevensToOnes[i][0]
  f.write("(constraint (= (rig_mimic {}) {}))\n".format(' '.join(map(lambda x: "{:.4f}".format(x), digit_two.flatten())), 'false'))

for i in range(num):
  digit_three = closestSevensToSevens[i][0]
  f.write("(constraint (= (rig_mimic {}) {}))\n".format(' '.join(map(lambda x: "{:.4f}".format(x), digit_three.flatten())), 'true'))

for i in range(num):
  digit_four = closestSevensToSevens[i][1]
  f.write("(constraint (= (rig_mimic {}) {}))\n".format(' '.join(map(lambda x: "{:.4f}".format(x), digit_four.flatten())), 'true'))

f.close()

# for i in range(25):
#     print(cosine_similarity(closestSevensToSevens[i][0].reshape(1,-1),closestSevensToSevens[i][1].reshape(1,-1)).flatten())
#     print(display_digit2(closestSevensToSevens[i][0]))
#     print(display_digit2(closestSevensToSevens[i][1]))

[[array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.   